In [ ]:
from pygom import DeterministicOde

6. And then initialize the model, which constructs our ODE system from all the information we have provided

In [ ]:
model = DeterministicOde(stateList, paramList, ode=odeList)

That is all the information required to define a simple SIR model. We can verify the model using

In [ ]:
model.get_ode_eqn()

where we can see the equations corresponding to their respective $S,I$ and $R$ state. 

```{note}
The ordering of the equations is in the standard $S,I,R$ sequence because of how we defined the states initially. 
```

We can rearrange the state list,

In [ ]:
# now we are going to define in a different order. note that the output ode changed with the input state
stateList = ['R', 'S', 'I']

model = DeterministicOde(stateList, paramList, ode=odeList)

model.get_ode_eqn()


and find that the set of ODEs comes out in the order that we specified. 

```{tip}
In addition to showing the equation form of the ODEs, we can also display them as either symbols or latex code, which can save some extra typing when porting the equations to another document.
```

In [ ]:
model.print_ode()

In [ ]:
model.print_ode(True)

Here the SIR model was provided to PyGOM as a set ODEs by using the {class}`.Transition` to define them.  
We have also provided the capability to obtain a *best guess* transition matrix when only the ODEs are available. See the section {doc}`unrollOde` for more information, and in particular {doc}`unrollSimple` for the continuing demonstration of the SIR model.

## Extracting model information

We may wish to determine if the set of ODEs are linear. 

In [ ]:
model.linear_ode()

Since we know that the SIR model is not linear, we may want to have a look at the Jacobian.


In [ ]:
model.get_jacobian_eqn()

Or maybe we want to know the gradient.

In [ ]:
model.get_grad_eqn()

```{Warning}
Invoking the functions that compute the derivatives, $f(x)$, `model.ode()` or `model.jacobian()`, will return an error

These functions are used to solve the ODEs numerically, and require values of initial state values, time, and parameter values.
```

For setting initial conditions, the numeric values of the states **must** be set in the same order that list of states appear. We can use the following to check the state ordering, as well as displaying all of the states that we have included.

In [ ]:
model.state_list

#TODO unsure if this is needed

There is currently no mechanism to set the numeric initial conditions the states when the states are defined. This is because of an implementation issue with external package, such as solving an initial value problem.

## Initial value problem

By setting the state initial conditions, time, and parameters, we can evaluate our model.

1. Define the model parameters. (We can call the parameters to check what we must provide.)

In [ ]:
model.parameters

In [ ]:
paramEval = [('beta',0.5), ('gamma',1.0/3.0)]

model.parameters = paramEval

model.parameters

2. Provide initial conditions for the states.

In [ ]:
initialState = [0, 1, 1.27e-6]
    
model.ode(state=initialState, t=1)


3. Implement an ODE solver.

We are well equipped to solve an initial value problem, using the standard numerical integrator such as `odeint <scipy.integrate.odeint>` from `scipy.integrate`. We also used `matplotlib.pyplot` for plotting and `linspace <numpy.linspace>` to create the time vector.

In [ ]:
import scipy.integrate
import numpy

t = numpy.linspace(0, 150, 100)

solution = scipy.integrate.odeint(model.ode, initialState, t)

We can plot our solution to observe a standard SIR shape.

In [ ]:
import matplotlib.pyplot as plt

plt.figure()

plt.plot(t, solution[:,0], label='R')

plt.plot(t, solution[:,1], label='S')

plt.plot(t, solution[:,2], label='I')

plt.xlabel('Time')

plt.ylabel('Population proportion')

plt.title('Standard SIR model')

plt.legend(loc=0)

#@savefig sir_plot.png In 

plt.show()

#plt.close()


Alternatively, we can integrate and plot via the **ode** object which we initialized earlier.

In [ ]:
model.initial_values = (initialState, t[0])

model.parameters = paramEval

solution = model.integrate(t[1::])

model.plot()

We could solve the ODEs above using the Jacobian as well. Unfortunately, it does not help because the number of times the Jacobian was evaluated was zero, as expected given that our set of equations are not stiff.

In [ ]:
#TODO what does this show?
%timeit solution1, output1 = scipy.integrate.odeint(model.ode, initialState, t, full_output=True)


In [ ]:

%timeit solution2, output2 = scipy.integrate.odeint(model.ode, initialState, t, Dfun=model.jacobian, mu=None, ml=None, full_output=True)


In [ ]:

%timeit solution3, output3 = model.integrate(t, full_output=True)

It is important to note that we return our Jacobian as a dense square matrix. Hence, the two argument (mu,ml) for the ODE solver was set to `None` to let it know the output explicitly.

## Solving the forward sensitivity equation

The sensitivity equations are also solved as an initial value problem. Let us redefine the model in the standard SIR order and we solve it with the sensitivity all set at zero, i.e. we do not wish to infer the initial value of the states.

In [ ]:
stateList = ['S', 'I', 'R']

model = DeterministicOde(stateList, paramList, ode=odeList)

initialState = [1, 1.27e-6, 0]

paramEval = [('beta', 0.5), ('gamma', 1.0/3.0)]

model.parameters = paramEval

solution = scipy.integrate.odeint(model.ode_and_sensitivity, numpy.append(initialState, numpy.zeros(6)), t)

In [ ]:
{
    "tags": [
        "hide-input",
    ]
}
f,axarr = plt.subplots(3,3);

f.text(0.5,0.975,'SIR with forward sensitivity solved via ode',fontsize=16,horizontalalignment='center',verticalalignment='top')

axarr[0,0].plot(t, solution[:,0])

axarr[0,0].set_title('S')

axarr[0,1].plot(t, solution[:,1])

axarr[0,1].set_title('I')

axarr[0,2].plot(t, solution[:,2]);

axarr[0,2].set_title('R')

axarr[1,0].plot(t, solution[:,3])

axarr[1,0].set_title(r'state S parameter $beta$')

axarr[2,0].plot(t, solution[:,4])

axarr[2,0].set_title(r'state S parameter $gamma$')

axarr[1,1].plot(t, solution[:,5])

axarr[1,1].set_title(r'state I parameter $beta$')

axarr[2,1].plot(t, solution[:,6])

axarr[2,1].set_title(r'state I parameter $gamma$')

axarr[1,2].plot(t, solution[:,7])

axarr[1,2].set_title(r'state R parameter $beta$')

axarr[2,2].plot(t, solution[:,8])

axarr[2,2].set_title(r'state R parameter $gamma$')

plt.tight_layout()

plt.show()

This concludes the introductory example and we will be moving on to look at parameter estimation next in {doc}`estimate1` and the most important part in terms of setting up the ODE object; defining the equations in various different ways in {doc}`transition`.